In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/dianfei/catch_robot_v2")

In [1]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
from sensor_msgs.msg import Image
import serial
import cv2
import math
import imutils
import time
from math import pi
import numpy as np
import threading
from collections import Counter
import os
import sys


# cap = None
# start,end = 0.0,0.0
# def nothing(x):
#     pass
# WindowName = 'result'
# cv2.namedWindow(WindowName, cv2.WINDOW_KEEPRATIO)  # 建立空窗口
# cv2.createTrackbar('Bl', WindowName, 0, 255, nothing)  # 创建滑动条
# cv2.createTrackbar('Gl', WindowName, 0, 255, nothing)  # 创建滑动条
# cv2.createTrackbar('Rl', WindowName, 0, 255, nothing)  # 创建滑动条
# cv2.createTrackbar('Bh', WindowName, 0, 255, nothing)  # 创建滑动条
# cv2.createTrackbar('Gh', WindowName, 0, 255, nothing)  # 创建滑动条
# cv2.createTrackbar('Rh', WindowName, 0, 255, nothing)  # 创建滑动条
# cv2.createTrackbar('iterations', WindowName, 0, 20, nothing)  # 创建滑动条


In [2]:

class Image_Receiver(Node):
    def __init__(self, name, topic="Image", parameter_overrides=[]):
        super().__init__(name, parameter_overrides=parameter_overrides)
        self.create_subscription(Image, topic, self.image_receive, 1)
        self.image = None

    def image_receive(self, msg):
        self.get_logger().info("Receive Image")
        frame = np.frombuffer(msg.data, dtype=np.uint8).reshape(
            msg.height, msg.width, -1
        )
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # frame = cv2.flip(frame, -1)
        self.image = frame

    def getImage(self):
        self.image = None
        rclpy.spin_once(self, timeout_sec=5)
        return self.image


def forecast_dis(dis, h, fruit_Z):
    # 抓水果时预测距离
    # dis为物体距离相机中心距离，h为框的长或高

    px = 0.0014
    f = 1.8
    dis = abs(dis)
    if dis == 0:
        deep = int(f * fruit_Z / (h * px))
        move = 0
    elif dis > h:
        dy = dis - h / 2
        angle_a = math.atan(dy * px / f)
        angle_b = math.atan(dis * px / f)
        c = math.sin(pi / 2 - angle_b) * fruit_Z / math.sin(angle_b - angle_a)
        y = c * math.sin(angle_a)
        move = int(y + fruit_Z)
        # deep = int(abs(c * math.cos(angle_a)))
        deep = int(f * move / (dis * px))

    elif dis < h:
        dy = dis + h / 2
        angle_a = math.atan(dy * px / f)
        angle_b = math.atan(dis * px / f)
        c = math.sin(pi / 2 - angle_a) * fruit_Z / math.sin(angle_a - angle_b)
        y = c * math.sin(angle_b)
        move = int(y)
        # deep = int(abs(c * math.cos(angle_b)))
        deep = int(f * move / (dis * px))

    else:
        move = fruit_Z
        deep = fruit_Z * f / (dis * px)

    return move, deep/2

def measure_dis(x_distance, y_distance, w, h, mode="fruit"):

    if mode == "fruit":
        fruit_w = 38
        fruit_h = 38
    else:
        fruit_w = 38
        fruit_h = 26
    x_move, x_deep = forecast_dis(x_distance, w, fruit_w)
    y_move, y_deep = forecast_dis(y_distance, h, fruit_h)
    if x_distance > 0:      # 往右走为正
        x_move = x_move
    elif x_distance < 0:
        x_move = - x_move
    else:
        x_move = 0
    if y_distance > 0:      # 往上动为正
        y_move = y_move
    elif y_distance < 0:
        y_move = -y_move
    else:
        y_move = 0

    # if x_distance > 0:
    #     print(f"向右移动{x_move}，x方向深度为{x_deep}")
    # elif x_distance < 0:
    #     print(f"向左移动{x_move}，x方向深度为{x_deep}")
    # else:
    #     print(f"在正中，x方向深度为{x_deep}")
    # if y_distance > 0:
    #     print(f"向下移动{y_move}，y方向深度为{y_deep}")
    # elif y_distance < 0:
    #     print(f"向上移动{y_move}，y方向深度为{y_deep}")
    # else:
    #     print(f"在正中，y方向深度为{y_deep}")
    # print("deep:", (x_deep + y_deep) / 2)

    return x_move, y_move, (x_deep + y_deep)/2 - 50

class ColorHandler:
    # 基于lab —— 测试后lab效果最好
    colors = [
        ([116, 140, 152], [213, 190, 199]),  # 红色/橙色
        ([27, 100, 137], [152, 119, 166]),  # 绿色
    ]  # 需要检测到的颜色
    filtered_colors = [
        ([0, 110, 168], [255, 148, 201])
    ]  # 需要过滤并去掉的颜色，如果有的话直接放进去lab格式的上下界即可

    colorNames = [
        "red",  # ripe
        "green"  # unripe
    ]

    def filter_color(self, blurred):
        truemask = np.zeros(blurred.shape[:2], dtype="uint8")
        filtered_mask = np.zeros(blurred.shape[:2], dtype="uint8")
        for (lower, upper) in self.colors:
            lower = np.array(lower, dtype='uint8')
            upper = np.array(upper, dtype='uint8')
            mask = cv2.inRange(blurred, lower, upper)
            truemask = cv2.bitwise_or(truemask, mask)

        for (lower, upper) in self.filtered_colors:
            lower = np.array(lower, dtype='uint8')
            upper = np.array(upper, dtype='uint8')
            mask = cv2.inRange(blurred, lower, upper)
            filtered_mask = cv2.bitwise_or(filtered_mask, mask)

        truemask = cv2.bitwise_or(truemask, filtered_mask)
        truemask = cv2.bitwise_xor(truemask, filtered_mask)
        color_filtered_image = cv2.bitwise_and(blurred, blurred, mask=truemask)
        return color_filtered_image

    def detect_color(self, image, cX, cY):  # cX和cY是轮廓检测到的对象的中心坐标
        mask = np.zeros(image.shape[:2], dtype="uint8")
        cv2.circle(mask, (cX, cY), 5, 255, -1)

        mask = cv2.erode(mask, None, iterations=2)  # 对上文得到的边缘进行erode，能成功模糊掉规定区域之外的部分，使得mask能成功保留下contour内区域
        mean = cv2.mean(image, mask=mask)[:3]  # 求image由mask罩起来的区域内的颜色平均值
        i = 0
        for (l, r) in self.colors:
            if mean[0] >= l[0] and mean[1] >= l[1] and mean[2] >= l[2] and mean[0] <= r[0] and mean[1] <= r[1] and mean[2] <= r[2]:
                break
            i += 1
        if i >= len(self.colorNames):
            return None
        return self.colorNames[i]

color_handler = ColorHandler()

def find_obj_bo_filter_color(image):  # bo 是 based on 的意思
    image = cv2.medianBlur(image, 3)		# 中值滤波的效果个人感觉用起来比高斯滤波好一点
    blurred = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    filtered = color_handler.filter_color(blurred)
    gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]      # 因为用颜色过滤过了，所以二值图阈值只要不会太高就行
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # 去边框
    maskedge_x = 0
    maskedge_y = 0
    maskedge_w = 640
    maskedge_h = 480
    cnts_in_range = []
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        #print(x, y, w, h)
        if not (abs(x - maskedge_x) <= 1 or abs(y - maskedge_y) <= 1 or abs(x + w - maskedge_x - maskedge_w) <= 1 or abs(y + h - maskedge_y - maskedge_h) <= 1):
            cnts_in_range.append(c)
    color = None
    c = None    # 初始化
    if len(cnts_in_range) > 0:

        c = max(cnts_in_range, key=cv2.contourArea)

        M = cv2.moments(c)
        if not M["m00"] == 0 and cv2.contourArea(c) > 4000:     # 保证得有轮廓，且面积太小的轮廓不要
            cX = int((M["m10"] / M["m00"]))
            cY = int((M["m01"] / M["m00"]))

            # 采摘用
            color = color_handler.detect_color(blurred, cX, cY)

            cv2.drawContours(image, [c], -1, (0, 0, 255), 2)
            x, y, w, h = cv2.boundingRect(c)
            # x, y, w, h = cv2.boundingRect(obj_contour)
            if w < 20 or h < 20:
                return image, color, None
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image,
                        f'{color}', (int(x), int(y - 2)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.60, (0, 0, 255),
                        thickness=1)
    #return filtered, color, c
        return image, color, c
    else:
        return image, color, None


class Camera:
    def __init__(self, video_ser, system_type="Linux"):
        self.cap = None
        self.video_ser = video_ser
        self.system_type = system_type
        self.init_camera()
        self.frame = None
        self.tickle = 40
        self.is_stop = False
        self.W = 640
        self.H = 480
        self.count = 0

    def init_camera(self):
        if self.system_type == "Windows":
            self.cap = cv2.VideoCapture(self.video_ser)
        else:
            part = "/dev/"
            self.cap = cv2.VideoCapture(part+self.video_ser)

    def start(self):
        t = threading.Thread(target=self.read, daemon=True)
        t.start()

    def read(self):
        while not self.is_stop:
            ret, frame = self.cap.read()
            if ret:
                self.frame = frame
            time.sleep(1 / self.tickle)
        self.cap.release()

    def stop(self):
        self.is_stop = True

    def get_frame(self):
        global Img_Receiver
        frame = Img_Receiver.getImage()
        if frame is not None:
            self.w = frame.shape[1]
            self.h = frame.shape[0]
            return frame
        else:
            self.count += 1
            print("未接收到图像")
            if self.count > 5:
                self.count = 0
                self.reboot_camera()
            return None

    def reboot_camera(self):
        # cmd = "ps aux | grep open_camera"
        # output = os.popen(cmd).readlines()
        # origin = "python3"
        # origin1 = "open_camera.py"
        # for i in output:
        #     i = i.strip()
        #     if origin in i:
        #         if origin1 in i:
        #             result = re.search(r"\b\d+\b", i)[0]
        #             kill_cmd = f"kill -9 {result}"
        #             print(kill_cmd)
        #             os.system(kill_cmd)
        #             print("杀死摄像头程序")
        # time.sleep(0.3)
        camera_thread = threading.Thread(target=self.restart)
        camera_thread.start()

    def restart(self):
        print("重新开启摄像头程序")
        restart_camera_cmd = "python3 /home/dianfei/catch_robot/open_camera.py"
        os.system(restart_camera_cmd)

    def detect(self, mode="fruit"):
        color_list = []
        result_list = []
        for i in range(10):

            frame, color, obj_contour = find_obj_bo_filter_color(self.get_frame())
            cv2.imshow("frame1", frame)
            cv2.waitKey(0)



In [3]:
print("开启摄像头...")
rclpy.init()
Img_Receiver = Image_Receiver("Image_Receiver", topic="/dev/video_left")
camera = Camera("video_left" , system_type="Linux")
# camera.init_camera()
# camera.start()
print("摄像头已开启")

开启摄像头...
摄像头已开启


[ERROR:0@40.380] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /dev/video_left in function 'icvExtractPattern'




In [4]:
def get_ordinary(pwm1, pwm2, pwm3, pwm4):
    l0 = 75
    l1 = 260
    l2 = 200
    l3 = 150
    # l0 = 40
    # l1 = 105
    # l2 = 75
    # l3 = 200
    # 计算每个舵机角度
    theta1 = (pwm1 - 1500) * 270 / 2000
    theta2 = -(pwm2 - 1500) * 270 / 2000
    theta3 = (pwm3 - 1500) * 270 / 2000
    theta4 = (pwm4 - 1500) * 270 / 2000
    # print(f"Theta1: {theta1}, Theta2: {theta2}, Theta3: {theta3}, Theta4: {theta4}")

    # if theta1 < -45 or theta1 > 135:
    #     return None
    theta4 = -theta4
    theta2 += 90
    angle = theta4 + theta2 - theta3
    theta3 = 180 - theta3

    theta1 = math.radians(theta1)
    theta2 = math.radians(theta2)
    theta3 = math.radians(theta3)
    theta4 = math.radians(theta4)
    # angle = math.radians(angle)
    aaa = math.cos(theta3)
    yz2 = -aaa * 2 * l1 * l2 + l1 * l1 + l2 * l2    # yz2为y2+z2
    bbb = (yz2 + l1 * l1 - l2 * l2) / (2 * l1 * math.sqrt(yz2))
    zf_ccc = theta2 - math.acos(bbb)
    # print(f"zf_ccc: {zf_ccc}")
    y = math.cos(zf_ccc) * math.sqrt(yz2)
    z = math.sqrt(yz2 - y * y)
    # print(z, l0)
    # print(l3 * math.sin(angle * pi / 180))

    y = y + l3 * math.cos(angle * pi / 180)    # 此时y为原始坐标x2+y2
    x = y * math.sin(theta1)
    y = y * math.cos(theta1)
    # if x > 0:
    if zf_ccc < 0:
        z = -z
    z = z + l0 + l3 * math.sin(angle * pi / 180)


    # print(f"y: {y}, z: {z},  x: {x}")

    X = int(x)
    Y = int(y)
    Z = int(z)
    angle = int(angle)
    return [X, Y, Z, angle]

class Arm:
    def __init__(self, direction="left"):
        # self.arm_ser: serial.Serial
        self.arm_pwm = {
            "0": 1500,
            "1": 1500,
            "2": 1500,
            "3": 1500,
            "4": 1500,
            "5": 1500,
        }
        self.X = 0
        self.Y = 0
        self.Z = 0
        self.angle = 0
        self.is_move = None
        self.count = 0
        self.is_close = False
        self.num = 0
        self.direction = direction
        self.start_aim_action = (300, 0, -30, 0)

    def init(self, Option) -> bool:
        try:
            self.arm_ser = serial.Serial(
                Option["port"], Option["baud_rate"]
            )
            if not self.arm_ser.is_open:
                self.arm_ser.open()  # 打开串口
            print("成功打开机械臂串")
            self.arm_ser.flushInput()
            return True
        except:
            print("机械臂串口打开失败")
            return False

    def start_aim(self):
        return self.send_ordinate(*self.start_aim_action)

    def receive(self):
        try:
            if self.arm_ser.in_waiting > 0:
                message = self.arm_ser.readline().decode('utf-8')
                message = message.strip()

                if message[0] == "@":
                    data = message.split('|')
                    index = data[0][1]
                    self.arm_pwm[index] = int(data[1])
                elif message[0] == "|":
                    if message[1] == "1":
                        self.is_move = True
                        # print("机械臂能到达坐标点")
                    elif message[1] == "0":
                        self.is_move = False

        except Exception as e:
            self.num += 1
            print(e)
            print("机械臂串口接收失败")

    def Receiver(self):
        while True:
            self.receive()
            if self.is_close or self.num > 10:
                break

    def rotate(self, angle):  # 机械臂想要旋转的角度
        self.get_cur_pwm()
        time.sleep(0.1)
        pwm0 = self.arm_pwm["0"]
        pwm0 = pwm0 + int(angle * 2000 / 270)
        # print('pwm0', pwm0)
        if pwm0 < 500:
            pwm0 = 500
        if pwm0 > 2500:
            pwm0 = 2500

        self.send_pwm(0, int(pwm0))

    def reset(self):
        self.arm_ser.write(("$DJR!\r\n").encode('utf-8'))

    def reset_except_claw(self, times=1500):
        command = "{" + "#000P1500T{times}!#001P1500T{times}!#002P1500T{times}!#003P1500T{times}!#004P1500T{times}!".format(
            times=times) + "}"
        self.arm_ser.write(command.encode('utf-8'))

    def put_down(self, times=1500):
        command = "{" + "#000P1500T{times}!#001P1400T{times}!#002P0900T{times}!#003P1300T{times}!#004P1500T{times}!".format(
            times=times) + "}"
        self.arm_ser.write(command.encode('utf-8'))
        time.sleep(1.5)
        self.send_pwm(5, 1500)

    def up(self, up_dis):
        x, y, z, angle = self.get_ordinate()
        self.send_ordinate(x, y, z + up_dis, angle, mode=1, times=1000)

    def send_pwm(self, index: int, pwm: int, times=1000):
        self.arm_pwm[str(index)] = pwm
        message = f"#00{index}P{pwm}T{times}!\r\n"
        self.arm_ser.write(message.encode('utf-8'))

    def send_ordinate_arr(self, x, y, z, angle, mode=0, times=1500):

        self.arm_ser.write(
            (f"$KMS:{int(x)},{int(y)},{int(z)},{times},{int(angle)},{mode}!\r\n").encode(encoding="UTF-8"))
        print(f"机械臂往 x：{x}， y：{y}， z：{z}， angle：{angle} 移动")

    def send_ordinate(self, x, y, z, angle, mode=0, times=1500, pos_ward=False):
        self.is_move = None
        self.send_ordinate_arr(x, y, z, angle, mode=mode, times=times)
        self.count = 0
        start_time = time.time()
        while True:
            time.sleep(0.05)
            cur_time = time.time()
            if cur_time - start_time > 4:
                print("未收到机械臂返回的结果")
                return False

            if self.is_move:
                time.sleep(1)
                print("机械臂成功到达")
                self.is_move = None
                return True

            elif self.is_move is False:
                self.count += 1
                if mode == 0:
                    if self.count < 5:
                        if pos_ward:
                            self.send_ordinate_arr(x, y, z, angle + self.count * 5, mode=mode, times=times)
                        else:
                            self.send_ordinate_arr(x, y, z, angle - self.count * 5, mode=mode, times=times)
                    else:
                        self.is_move = None
                        return False
                elif mode == 1:
                    if self.count < 5:
                        if pos_ward:
                            self.send_ordinate_arr(x + self.count * 5, y, z, angle, mode=mode, times=times)
                        else:
                            self.send_ordinate_arr(x - self.count * 5, y, z, angle, mode=mode, times=times)
                    else:
                        self.is_move = None
                        return False


    def get_pwm(self):
        return self.arm_pwm

    # def update(self):
    #     self.get_cur_pwm()
    #     self.get_ordinate()
    #     # thread = threading.Timer(3, self.update)
    #     # thread.start()

    def get_cur_pwm(self):
        self.arm_ser.write(("@|\r\n").encode(encoding="UTF-8"))

    def get_ordinate(self):
        self.get_cur_pwm()
        time.sleep(0.2)
        pwm0 = self.arm_pwm['0']
        pwm1 = self.arm_pwm['1']
        pwm2 = self.arm_pwm['2']
        pwm3 = self.arm_pwm['3']
        result = get_ordinary(pwm0, pwm1, pwm2, pwm3)
        print(f"当前机械臂位置为 x：{result[0]}， y：{result[1]}， z：{result[2]}， angle：{result[3]}")
        return result

    def get_data(self):
        return self.X, self.Y, self.Z, self.angle

    def run_thread(self):
        arm_receive_thread = threading.Thread(target=self.Receiver, daemon=True)
        # arm_update_thread = threading.Timer(1, self.update)
        arm_receive_thread.start()
        # arm_update_thread.start()

    def close(self):
        self.arm_ser.close()
        self.is_close = True
        print("机械臂串口关闭")



In [5]:
Option = {
    "arm_ser": {"port": "/dev/ttyUSB0", "baud_rate": 115200}
}
print("初始化机械臂...")
arm = Arm()
arm.init(Option["arm_ser"])
arm.run_thread()
print("机械臂已初始化")

初始化机械臂...
成功打开机械臂串
机械臂已初始化


In [16]:
arm.send_ordinate(0, 250, 350, angle=-5, times=1500, mode=0)


机械臂往 x：0， y：250， z：350， angle：-5 移动
机械臂成功到达


True

In [12]:
arm.reset()

In [10]:
arm.send_ordinate(0, 350, 200, angle=-95, times=1500, mode=0)

机械臂往 x：0， y：350， z：200， angle：-95 移动
机械臂成功到达


True

In [7]:
camera.detect()

[INFO] [1721548466.176091288] [Image_Receiver]: Receive Image
[INFO] [1721548471.579547555] [Image_Receiver]: Receive Image
[INFO] [1721548475.195570946] [Image_Receiver]: Receive Image
[INFO] [1721548480.215797337] [Image_Receiver]: Receive Image
[INFO] [1721548481.348309745] [Image_Receiver]: Receive Image
[INFO] [1721548482.078554094] [Image_Receiver]: Receive Image
[INFO] [1721548482.780308509] [Image_Receiver]: Receive Image
[INFO] [1721548483.334356427] [Image_Receiver]: Receive Image
[INFO] [1721548483.765983370] [Image_Receiver]: Receive Image
[INFO] [1721548484.211001217] [Image_Receiver]: Receive Image


In [8]:
for i in range(10):
    frame = camera.get_frame()
    cv2.imshow("frame", frame)
    cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] [1721545830.184057922] [Image_Receiver]: Receive Image
[INFO] [1721545835.346523300] [Image_Receiver]: Receive Image
[INFO] [1721545836.408624634] [Image_Receiver]: Receive Image
[INFO] [1721545837.274129218] [Image_Receiver]: Receive Image
[INFO] [1721545837.986659414] [Image_Receiver]: Receive Image
[INFO] [1721545839.020678332] [Image_Receiver]: Receive Image
[INFO] [1721545839.811843314] [Image_Receiver]: Receive Image
[INFO] [1721545840.514881963] [Image_Receiver]: Receive Image
[INFO] [1721545841.062847904] [Image_Receiver]: Receive Image
[INFO] [1721545841.419007713] [Image_Receiver]: Receive Image


In [7]:
while True:
    time.sleep(0.05)
    global start,end
    frame = camera.get_frame()

    # 获取滑动条值
    Bl = cv2.getTrackbarPos('Bl', WindowName)
    Gl = cv2.getTrackbarPos('Gl', WindowName)
    Rl = cv2.getTrackbarPos('Rl', WindowName)
    Bh = cv2.getTrackbarPos('Bh', WindowName)
    Gh = cv2.getTrackbarPos('Gh', WindowName)
    Rh = cv2.getTrackbarPos('Rh', WindowName)
    ite = cv2.getTrackbarPos('iterations', WindowName)


    #色域设置
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_color = np.array([Bl, Gl, Rl])
    upper_color = np.array([Bh, Gh, Rh])

    #开运算
    hsv_image = cv2.erode(hsv_image, np.ones((3,3),np.uint8), iterations=ite)
    hsv_image = cv2.dilate(hsv_image, np.ones((3,3),np.uint8), iterations=ite)

    #模糊处理
    hsv_image = cv2.blur(hsv_image, (9, 9))

    #二值转换，进行颜色分割---》把色域内的像素点设为白色，其余像素点设为黑色
    mask = cv2.inRange(hsv_image, lower_color, upper_color)


    #获取色块轮廓（cv2.findContours()函数返回的轮廓列表是按轮廓大小排序的）
    contours,hierarchy= cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    biggest_area = 0
    if contours :
        for contour in contours:#筛选出目标色块
            x, y, w, h = cv2.boundingRect(contour)
            area = cv2.contourArea(contour)
            if area > biggest_area:
                biggest_area = area
            # print(str(biggest_area) + " _____ " + str(x + w / 2) + " _____ " + str(y + h / 2))
            #绘制矩形框
            cv2.rectangle(hsv_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.namedWindow("hsv_image", cv2.WINDOW_KEEPRATIO)
    cv2.namedWindow("mask", cv2.WINDOW_KEEPRATIO)

    cv2.imshow("hsv_image", hsv_image)
    cv2.imshow("mask", mask)
    # 按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

[INFO] [1722595898.479064705] [Image_Receiver]: Receive Image
[INFO] [1722595928.207132957] [Image_Receiver]: Receive Image
[INFO] [1722595928.400200204] [Image_Receiver]: Receive Image
[INFO] [1722595928.526991242] [Image_Receiver]: Receive Image
[INFO] [1722595928.641641244] [Image_Receiver]: Receive Image
[INFO] [1722595928.785591524] [Image_Receiver]: Receive Image
[INFO] [1722595928.898355552] [Image_Receiver]: Receive Image
[INFO] [1722595929.018724105] [Image_Receiver]: Receive Image
[INFO] [1722595929.129952265] [Image_Receiver]: Receive Image
[INFO] [1722595929.244375565] [Image_Receiver]: Receive Image
[INFO] [1722595929.365003465] [Image_Receiver]: Receive Image
[INFO] [1722595929.480499177] [Image_Receiver]: Receive Image
[INFO] [1722595929.589611126] [Image_Receiver]: Receive Image
[INFO] [1722595929.703164902] [Image_Receiver]: Receive Image
[INFO] [1722595929.810896073] [Image_Receiver]: Receive Image
[INFO] [1722595929.928995008] [Image_Receiver]: Receive Image
[INFO] [

KeyboardInterrupt: 

In [18]:

arm.reset()

In [9]:
while True:
    time.sleep(0.05)
    global start,end
    frame = camera.get_frame()

    # 获取滑动条值
    Bl = cv2.getTrackbarPos('Bl', WindowName)
    Gl = cv2.getTrackbarPos('Gl', WindowName)
    Rl = cv2.getTrackbarPos('Rl', WindowName)
    Bh = cv2.getTrackbarPos('Bh', WindowName)
    Gh = cv2.getTrackbarPos('Gh', WindowName)
    Rh = cv2.getTrackbarPos('Rh', WindowName)
    ite = cv2.getTrackbarPos('iterations', WindowName)


    #色域设置
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    lower_color = np.array([Bl, Gl, Rl])
    upper_color = np.array([Bh, Gh, Rh])

    #开运算
    hsv_image = cv2.erode(hsv_image, np.ones((3,3),np.uint8), iterations=ite)
    hsv_image = cv2.dilate(hsv_image, np.ones((3,3),np.uint8), iterations=ite)

    #模糊处理
    hsv_image = cv2.blur(hsv_image, (9, 9))

    #二值转换，进行颜色分割---》把色域内的像素点设为白色，其余像素点设为黑色
    mask = cv2.inRange(hsv_image, lower_color, upper_color)


    #获取色块轮廓（cv2.findContours()函数返回的轮廓列表是按轮廓大小排序的）
    contours,hierarchy= cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    biggest_area = 0
    if contours :
        for contour in contours:#筛选出目标色块
            x, y, w, h = cv2.boundingRect(contour)
            area = cv2.contourArea(contour)
            if area > biggest_area:
                biggest_area = area
            # print(str(biggest_area) + " _____ " + str(x + w / 2) + " _____ " + str(y + h / 2))
            #绘制矩形框
            cv2.rectangle(hsv_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.namedWindow("hsv_image", cv2.WINDOW_KEEPRATIO)
    cv2.namedWindow("mask", cv2.WINDOW_KEEPRATIO)

    cv2.imshow("hsv_image", hsv_image)
    cv2.imshow("mask", mask)
    # 按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[INFO] [1721547052.950879883] [Image_Receiver]: Receive Image
[INFO] [1721547053.909725048] [Image_Receiver]: Receive Image
[INFO] [1721547054.095825829] [Image_Receiver]: Receive Image
[INFO] [1721547054.276311249] [Image_Receiver]: Receive Image
[INFO] [1721547054.429733465] [Image_Receiver]: Receive Image
[INFO] [1721547054.596777902] [Image_Receiver]: Receive Image
[INFO] [1721547054.752899669] [Image_Receiver]: Receive Image
[INFO] [1721547054.898414392] [Image_Receiver]: Receive Image
[INFO] [1721547055.085974873] [Image_Receiver]: Receive Image
[INFO] [1721547055.232081569] [Image_Receiver]: Receive Image
[INFO] [1721547055.419286055] [Image_Receiver]: Receive Image
[INFO] [1721547055.602261530] [Image_Receiver]: Receive Image
[INFO] [1721547055.782947243] [Image_Receiver]: Receive Image
[INFO] [1721547056.028690036] [Image_Receiver]: Receive Image
[INFO] [1721547056.183354158] [Image_Receiver]: Receive Image
[INFO] [1721547056.323397592] [Image_Receiver]: Receive Image
[INFO] [

In [19]:
cv2.destroyAllWindows()